# BIL ETL - PRISM

## Importing Packages

In [42]:
import arcpy
import requests
import os
import zipfile
import io

## Creating a Working Folder

In [43]:
Prism_working = r'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM'

In [44]:
arcpy.env.workspace = Prism_working
arcpy.env.workspace

'C:\\Users\\tande\\Documents\\GIS5571\\GIS5571 Lab 2\\PRISM'

## Calling in Data

In [45]:
Prism_base = r'https://prism.oregonstate.edu/fetchData.php'
Prism_params = r'type=all_bil&kind=normals&spatial=4km&elem=ppt&temporal=annual'
Prism_path = Prism_base + '?' + Prism_params

In [46]:
Prism_post_request = requests.post(Prism_path)
Prism_post_request

<Response [200]>

In [47]:
Prism_zipped = zipfile.ZipFile(
        io.BytesIO(
            Prism_post_request.content)
)

In [48]:
Prism_zipped.extractall(Prism_working)

## Create a Space Time Cube

In [49]:
# Ref https://www.esri.com/arcgis-blog/products/arcgis-pro/analytics/explore-your-raster-data-with-space-time-pattern-mining/

In [50]:
#Create a mosaic to combine multiple rasters
arcpy.management.CreateMosaicDataset(r"C:\Users\tande\Documents\ArcGIS\Projects\GIS5571_Lab2\GIS5571_Lab2.gdb", "Prism_Raster_Mosaic", 'GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', None, '', "NONE", None)

ExecuteError: ERROR 000258: Output C:\Users\tande\Documents\ArcGIS\Projects\GIS5571_Lab2\GIS5571_Lab2.gdb\Prism_Raster_Mosaic already exists
Failed to execute (CreateMosaicDataset).


In [ ]:
#Adding the rasters
arcpy.management.AddRastersToMosaicDataset("Prism_Raster_Mosaic", "Raster Dataset", r"'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_01_bil.bil';'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_02_bil.bil';'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_03_bil.bil';'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_04_bil.bil';'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_05_bil.bil';'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_06_bil.bil';'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_07_bil.bil';'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_08_bil.bil';'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_09_bil.bil';'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_10_bil.bil';'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_11_bil.bil';'C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\PRISM_ppt_30yr_normal_4kmM3_12_bil.bil'", "UPDATE_CELL_SIZES", "UPDATE_BOUNDARY", "NO_OVERVIEWS", None, 0, 1500, None, '', "SUBFOLDERS", "ALLOW_DUPLICATES", "NO_PYRAMIDS", "NO_STATISTICS", "NO_THUMBNAILS", '', "NO_FORCE_SPATIAL_REFERENCE", "NO_STATISTICS", None, "NO_PIXEL_CACHE", r"C:\Users\tande\AppData\Local\ESRI\rasterproxies\Prism_Raster_Mosaic")

In [ ]:
#Add time to the raster dataset

arcpy.management.CalculateField(r"Prism_Raster_Mosaic\Footprint", "Variable", '"PRISM"', "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")
arcpy.management.CalculateField(r"Prism_Raster_Mosaic\Footprint", "Timestamp", "DateAdd(Date(2020,0,1), $feature.OBJECTID-1, 'year')", "ARCADE", '', "DATE", "NO_ENFORCE_DOMAINS")
#Ref https://www.esri.com/arcgis-blog/products/arcgis-pro/analytics/explore-your-raster-data-with-space-time-pattern-mining/

In [ ]:
#Build a multidimentional raster layer

arcpy.md.BuildMultidimensionalInfo("Prism_Raster_Mosaic", "Variable", "Timestamp # #", None, "NO_DELETE_MULTIDIMENSIONAL_INFO")
arcpy.md.MakeMultidimensionalRasterLayer("Prism_Raster_Mosaic", "Prism_Raster_Mosaic_MultidimLayer", "PRISM", "ALL", None, None, '', '', '', None, '', '-125.020833333333 24.0624999997935 -66.4791666661985 49.9375000000005 GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "DIMENSIONS", None)

In [1]:
#Creating the Space Time Cube

arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer("Prism_Raster_Mosaic_MultidimLayer", r"C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\Prism_STC.nc", "ZEROS")

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Multidimensional Raster Layer: Dataset Prism_Raster_Mosaic_MultidimLayer does not exist or is not supported
WARNING 000725: Output Space Time Cube: Dataset C:\Users\tande\Documents\GIS5571\GIS5571 Lab 2\PRISM\Prism_STC.nc already exists.
Failed to execute (CreateSpaceTimeCubeMDRasterLayer).


## Export Animated STC

In [2]:
# Ref https://pro.arcgis.com/en/pro-app/latest/help/sharing/overview/gif-export.htm

# I was able to manually export the animated STC using the 'Export Movie' icon 
#but unfortunately was not able to do this via arcpy. 

#The saved GIF file can be found on GitHub